In [1]:
using MLDatasets
using Flux, Statistics
using Flux: onehotbatch, onecold
using BSON: @load

In [2]:
test_x, test_y = FashionMNIST.testdata()
test_x = [test_x[:, :, i] for i in 1:size(test_x,3)]

function mb(img, label, indexx) 
    mb_x = Array{Float32}(undef, size(img[1])..., 1, length(indexx)) # placeholder
    for i in 1:length(indexx)
        mb_x[:, :, :, i] = Float32.(img[indexx[i]]) # Asigns images into batches of 64
    end
    # This encodes labels into one-hot vector representation
    mb_y = onehotbatch(label[indexx], 0:9)
    return (mb_x, mb_y)
end

mb (generic function with 1 method)

In [3]:
ConvNet = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),
    Conv((3, 3), 32=>64, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(576, 72),
    x -> relu.(x),
    Dense(72, 10),
    x -> relu.(x),
    softmax)

Chain(Conv((3, 3), 1=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), #5, Dense(576, 72), #6, Dense(72, 10), #7, softmax)

In [34]:
@load "fmnist_flux_3.bson" (ConvNet)
# Flux.loadparams!(ConvNet)

In [35]:
mb_size = 64
testset = mb(test_x, test_y, 1:length(test_x))
accuracy(x, y) = mean(onecold(ConvNet(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [38]:
acc = accuracy(testset...)
print("The test accuracy is ", acc)

The test accuracy is 0.8149